<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7397a750cb001c7fd2276b6acb8c9feeed9412889574b6fd4dee2e8f0ccb3e40
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-13 13:10:15
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: -1.39 L (-0.98%)
Current PnL: -21.55 L (-14.36%)
CY Booked + Current PnL: -9.65 L (-6.43%)
-------------------
Total profit:  1.12 L
Total loss:  -22.67 L
-------------------
Total Booked + Current PnL: 17.33 L (14.07%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.5%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.76 L (63.43%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.64%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,59.0,X-LC,3.69,214172.0,9672.0,13621.0,0.10,4.73,6.36,11.39,10.0,0.71,1.54,14.10,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,94.02,42.0,M-SC,1.25,83611.0,-17166.0,17257.0,0.47,-17.03,20.64,0.09,245.0,-0.99,0.60,9.85,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-1.64,50.0,X-MC,1.04,206325.0,14583.0,23170.0,-2.95,7.61,11.23,19.69,99.0,0.63,1.49,14.92,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,46.0,X-LC,2.97,214141.0,3775.0,25012.0,-0.65,1.79,11.68,13.68,37.0,0.15,1.54,18.61,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195184.0,-4954.0,25842.0,-0.91,-2.48,13.24,10.44,4.0,-0.19,1.41,2.95,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIS,528.00,1994.82,38.0,H-SC,3.83,84532.0,-26500.0,50111.0,1.92,-23.87,59.28,21.26,156.0,-0.53,0.61,14.21,OX40N,NTT,MISC
36,INDIAMART,4810.62,-54.27,44.0,H-SC,8.52,127481.0,4145.0,127481.0,0.58,3.36,100.00,106.72,119.0,0.03,0.92,27.52,AR,ATH,MISC
76,TTKPRESTIG,770.00,94.02,42.0,M-SC,1.25,83611.0,-17166.0,17257.0,0.47,-17.03,20.64,0.09,245.0,-0.99,0.60,9.85,OX40N,NTT,DURABLES
42,JCHAC,2282.00,19769.57,48.0,M-SC,1.71,97065.0,-30740.0,30731.0,0.30,-24.05,31.66,-0.01,232.0,-1.00,0.70,9.91,OX40N,NTT,AC
5,ASIANPAINT,3465.66,-18.29,35.0,X-LC,5.47,206378.0,-45390.0,98607.0,0.21,-18.03,47.78,21.13,27.0,-0.46,1.49,11.01,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GREENPANEL,537.0,227.52,54.0,M-SC,4.48,146254.0,-36824.0,115804.0,-3.26,-20.11,79.18,43.14,230.0,-0.32,1.05,37.57,XY24,NTT,MISC
46,LAOPALA,464.0,100.59,35.0,H-SC,1.20,91726.0,-39379.0,90625.0,-3.04,-30.04,98.80,39.09,142.0,-0.43,0.66,18.14,AR,NTT,CERAMICS
81,VALIANTORG,838.0,-287.77,28.0,H-SC,2.35,104397.0,-59208.0,180523.0,-2.99,-36.19,172.92,74.15,139.0,-0.33,0.75,33.82,XR,NTT,CHEMICALS
83,VOLTAS,1530.0,-1.64,50.0,X-MC,1.04,206325.0,14583.0,23170.0,-2.95,7.61,11.23,19.69,99.0,0.63,1.49,14.92,XY25,NTT,AC
20,COFFEEDAY,80.0,-43.66,51.0,L-SC,29.07,82214.0,-31335.0,71386.0,-2.92,-27.60,86.83,35.27,268.0,-0.44,0.59,101.22,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.0,42.74,48.0,M-MC,12.59,225582.0,620.0,167043.0,-0.89,0.28,74.05,74.53,192.0,0.00,1.62,32.62,XY24,BTT,STEEL
54,RECLTD,446.0,45.36,43.0,M-LC,6.34,203500.0,330.0,41799.0,-0.79,0.16,20.54,20.74,55.0,0.01,1.47,5.71,XY25,NTT,FINANCE
85,WIPRO,420.0,-13.34,49.0,M-LC,5.83,151987.0,1042.0,108412.0,-1.43,0.69,71.33,72.51,53.0,0.01,1.09,6.75,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,63.0,H-SC,11.32,131517.0,-10962.0,30486.0,-0.14,-7.69,23.18,13.70,163.0,-0.36,0.95,55.23,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,94.02,42.0,M-SC,1.25,83611.0,-17166.0,17257.0,0.47,-17.03,20.64,0.09,245.0,-0.99,0.60,9.85,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,49.0,H-SC,1.98,221544.0,-48123.0,84453.0,-0.54,-17.85,38.12,13.47,138.0,-0.57,1.60,12.65,XY24,NTT,PAINTS
17,CERA,9475.0,-25.30,31.0,H-SC,1.94,136206.0,-39697.0,81724.0,-0.75,-22.57,60.00,23.89,149.0,-0.49,0.98,17.63,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.72,43.0,H-MC,6.81,101935.0,-29900.0,72119.0,-0.79,-22.68,70.75,32.03,98.0,-0.41,0.73,14.18,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,43.0,M-LC,6.34,203500.0,330.0,41799.0,-0.79,0.16,20.54,20.74,55.0,0.01,1.47,5.71,XY25,NTT,FINANCE
59,SAIL,228.00,42.74,48.0,M-MC,12.59,225582.0,620.0,167043.0,-0.89,0.28,74.05,74.53,192.0,0.00,1.62,32.62,XY24,BTT,STEEL
31,HINDZINC,730.22,32.99,64.0,M-LC,7.80,216760.0,11684.0,100880.0,-2.72,5.70,46.54,54.89,52.0,0.12,1.56,31.17,X5K,ATH,METALS
78,UNIONBANK,163.00,-13.18,56.0,M-LC,9.35,158043.0,17203.0,27294.0,-0.19,12.21,17.27,31.59,66.0,0.63,1.14,41.94,XY24,NTT,BANKS
85,WIPRO,420.00,-13.34,49.0,M-LC,5.83,151987.0,1042.0,108412.0,-1.43,0.69,71.33,72.51,53.0,0.01,1.09,6.75,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-25.47,65.0,H-MC,12.31,193819.0,14011.0,98402.0,-0.59,7.79,50.77,62.52,88.0,0.14,1.40,44.53,XR,NTT,BANKS
31,HINDZINC,730.22,32.99,64.0,M-LC,7.80,216760.0,11684.0,100880.0,-2.72,5.70,46.54,54.89,52.0,0.12,1.56,31.17,X5K,ATH,METALS
36,INDIAMART,4810.62,-54.27,44.0,H-SC,8.52,127481.0,4145.0,127481.0,0.58,3.36,100.00,106.72,119.0,0.03,0.92,27.52,AR,ATH,MISC
85,WIPRO,420.00,-13.34,49.0,M-LC,5.83,151987.0,1042.0,108412.0,-1.43,0.69,71.33,72.51,53.0,0.01,1.09,6.75,XR,NTT,IT
84,WHIRLPOOL,2270.00,-48.37,31.0,M-SC,2.91,87270.0,-4228.0,82976.0,-0.50,-4.62,95.08,86.07,223.0,-0.05,0.63,26.73,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,32.0,X-SC,37.76,52429.0,-12577.0,163505.0,-2.38,-19.35,311.86,232.18,198.0,-0.08,0.38,0.59,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.45,33.0,X-MC,0.90,229554.0,-18177.0,93245.0,-1.63,-7.34,40.62,30.30,92.0,-0.19,1.65,4.98,X40,ATH,ELECTRICAL
22,DABUR,735.00,-12.84,34.0,X-MC,3.56,235807.0,-12261.0,120662.0,-0.71,-4.94,51.17,43.70,102.0,-0.10,1.70,8.79,XY24,BTT,FMCG
5,ASIANPAINT,3465.66,-18.29,35.0,X-LC,5.47,206378.0,-45390.0,98607.0,0.21,-18.03,47.78,21.13,27.0,-0.46,1.49,11.01,X40,ATH,PAINTS
9,BAJAJHFL,181.50,-15.11,35.0,X-MC,5.14,178760.0,-21746.0,118893.0,-0.91,-10.85,66.51,48.45,90.0,-0.18,1.29,3.08,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,HAVELLS,2069.16,-7.45,33.0,X-MC,0.90,229554.0,-18177.0,93245.0,-1.63,-7.34,40.62,30.30,92.0,-0.19,1.65,4.98,X40,ATH,ELECTRICAL
83,VOLTAS,1530.00,-1.64,50.0,X-MC,1.04,206325.0,14583.0,23170.0,-2.95,7.61,11.23,19.69,99.0,0.63,1.49,14.92,XY25,NTT,AC
72,TATAMOTORS,1065.00,-23.48,38.0,X-LC,1.19,241177.0,-35346.0,145430.0,-2.14,-12.78,60.30,39.81,54.0,-0.24,1.74,15.59,XY24,NTT,AUTO
64,SIEMENS,4671.50,-4.75,39.0,H-LC,1.27,153150.0,-32945.0,80419.0,-2.68,-17.70,52.51,25.51,15.0,-0.41,1.10,12.86,AR,ATH,ELECTRICAL
33,ICICIGI,2260.25,-21.04,37.0,X-MC,1.65,135642.0,-491.0,31618.0,-1.60,-0.36,23.31,22.86,91.0,-0.02,0.98,12.21,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,32.0,X-SC,37.76,52429.0,-12577.0,163505.0,-2.38,-19.35,311.86,232.18,198.0,-0.08,0.38,0.59,XY24,NTT,MISC
82,VBL,671.64,-22.00,36.0,X-LC,10.50,280688.0,-35154.0,147810.0,-0.83,-11.13,52.66,35.67,5.0,-0.24,2.02,1.18,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.20,37.0,X-MC,7.46,221400.0,-41965.0,155312.0,-0.62,-15.93,70.15,43.04,84.0,-0.27,1.59,2.79,XY25,ATH,FMCG
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195184.0,-4954.0,25842.0,-0.91,-2.48,13.24,10.44,4.0,-0.19,1.41,2.95,X40,NTT,FMCG
9,BAJAJHFL,181.50,-15.11,35.0,X-MC,5.14,178760.0,-21746.0,118893.0,-0.91,-10.85,66.51,48.45,90.0,-0.18,1.29,3.08,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,32.0,X-SC,37.76,52429.0,-12577.0,163505.0,-2.38,-19.35,311.86,232.18,198.0,-0.08,0.38,0.59,XY24,NTT,MISC
55,RELAXO,1176.00,-40.56,36.0,X-SC,2.88,81084.0,-64076.0,140000.0,-0.76,-44.14,172.66,52.31,136.0,-0.46,0.58,8.54,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.87,39.0,X-SC,4.40,95782.0,-32888.0,78187.0,-1.28,-25.56,81.63,35.20,219.0,-0.42,0.69,9.94,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.16,50.0,X-SC,1.21,107764.0,5760.0,38170.0,-0.31,5.65,35.42,43.06,122.0,0.15,0.78,23.73,X40N,ATH,MISC
32,HONAUT,58357.33,-22.88,46.0,X-SC,2.19,108750.0,-19188.0,66327.0,-0.62,-15.00,60.99,36.84,143.0,-0.29,0.78,11.33,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.01,49.0,X-LC,12.18,283081.0,-62875.0,131123.0,-0.55,-18.17,46.32,19.73,1.0,-0.48,2.04,4.26,X40,ATH,IT
39,INFY,2275.00,-18.88,50.0,X-LC,7.64,316624.0,3736.0,167937.0,-1.87,1.19,53.04,54.87,3.0,0.02,2.28,7.92,X40,BTT,IT
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195184.0,-4954.0,25842.0,-0.91,-2.48,13.24,10.44,4.0,-0.19,1.41,2.95,X40,NTT,FMCG
82,VBL,671.64,-22.00,36.0,X-LC,10.50,280688.0,-35154.0,147810.0,-0.83,-11.13,52.66,35.67,5.0,-0.24,2.02,1.18,X40N,ATH,FMCG
1,ABB,7934.00,-41.03,42.0,H-LC,7.44,246432.0,-15187.0,134404.0,-1.00,-5.81,54.54,45.57,7.0,-0.11,1.77,4.86,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-43.66,51.0,L-SC,29.07,82214.0,-31335.0,71386.0,-2.92,-27.60,86.83,35.27,268.0,-0.44,0.59,101.22,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,42.0,H-SC,10.03,90360.0,-7620.0,29222.0,-1.18,-7.78,32.34,22.05,152.0,-0.26,0.65,31.34,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,107.70,63.0,H-SC,11.32,131517.0,-10962.0,30486.0,-0.14,-7.69,23.18,13.70,163.0,-0.36,0.95,55.23,OX40N,NTT,BANKS
25,GREENPANEL,537.00,227.52,54.0,M-SC,4.48,146254.0,-36824.0,115804.0,-3.26,-20.11,79.18,43.14,230.0,-0.32,1.05,37.57,XY24,NTT,MISC
60,SAMMAANCAP,326.00,-184.09,64.0,M-SC,16.37,145404.0,-3816.0,147992.0,-0.41,-2.56,101.78,96.62,208.0,-0.03,1.05,57.36,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-184.09,64.0,M-SC,16.37,145404.0,-3816.0,147992.0,-0.41,-2.56,101.78,96.62,208.0,-0.03,1.05,57.36,XY25,NTT,FINANCE
68,SURYODAY,240.00,64.47,60.0,H-SC,9.86,150307.0,-28764.0,94498.0,-2.42,-16.06,62.87,36.71,135.0,-0.30,1.08,49.13,XR,NTT,BANKS
31,HINDZINC,730.22,32.99,64.0,M-LC,7.80,216760.0,11684.0,100880.0,-2.72,5.70,46.54,54.89,52.0,0.12,1.56,31.17,X5K,ATH,METALS
63,SHALBY,327.00,1183.89,56.0,M-SC,22.36,174671.0,-6472.0,51947.0,0.00,-3.57,29.74,25.10,235.0,-0.12,1.26,39.98,XY24,NTT,HEALTHCARE
67,STARHEALTH,761.00,16.61,67.0,H-SC,7.83,259787.0,-21336.0,154210.0,-0.25,-7.59,59.36,47.26,144.0,-0.14,1.87,39.19,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.27
1,25,43.81
2,50,75.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.36
LC    33.98
MC    21.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.30
X40      14.73
XY25     11.95
XR       10.99
X40N      9.96
AR        8.18
OX40N     7.65
X200      1.77
X5K       1.56
SR        1.12
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.68
X-LC    23.06
X-MC    15.95
M-SC    12.68
M-LC     5.26
H-LC     4.64
X-SC     4.55
H-MC     3.79
M-MC     1.62
L-SC     1.45
L-LC     1.02
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.88,-6.45,41.53
IT,12.92,-17.13,80.56
FINANCE,12.02,-13.36,64.75
MISC,6.99,-18.54,84.22
BANKS,6.51,-10.54,68.69
PAINTS,5.58,-19.41,37.21
ELECTRICAL,5.30,-11.59,50.74
INSURANCE,3.82,-4.41,43.44
AUTO,3.50,-18.62,75.32


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3229914.0,22
XR,1382459.0,14
AR,1193663.0,9
X40,795962.0,10
X40N,644413.0,8
OX40N,584608.0,10
XY25,524409.0,8
SR,266107.0,2
X5K,100880.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3437286.0,25
M-SC,1281267.0,15
X-MC,1112886.0,11
X-LC,1039797.0,13
X-SC,549337.0,6
H-MC,353222.0,3
H-LC,294649.0,3
M-LC,278385.0,4
L-SC,259728.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1195583.0      6
           XR         814340.0      7
           AR         813392.0      5
M-SC       XY24       687073.0      6
X-MC       XY24       591709.0      4
X-LC       X40        493973.0      5
X-SC       X40N       307645.0      4
H-SC       OX40N      274219.0      4
           SR         266107.0      2
X-MC       X40        223802.0      4
X-LC       X40N       217875.0      3
           XY24       215006.0      2
H-LC       AR         214823.0      2
H-MC       XY24       182701.0      1
X-MC       XY25       178482.0      2
L-SC       XR         170381.0      2
M-MC       XY24       167043.0      1
M-SC       AR         165448.0      2
X-SC       XY24       163505.0      1
M-SC       OX40N      148923.0      4
           XY25       147992.0      1
           XR         131831.0      2
X-MC       X40N       118893.0      1
X-LC       XY25       112943.0      3
M-LC       XR         108412.0      1
           X5K        100880.0      1
H-MC       XR          98402.0      1
L-SC       OX40N       89347.0      1
H-LC       X200        79826.0      1
X-SC       X40         78187.0      1
H-SC       MH          73645.0      1
H-MC       OX40N       72119.0      1
L-MC       XR          59093.0      1
L-LC       XY25        43193.0      1
M-LC       XY25        41799.0      1
           XY24        27294.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 57.0 seconds
